In [1]:
# config and imports

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import zscore

DATA_PATH = "Zip_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv"
TARGET_STATE = "GA"
CUTOFF_COL = "2024-02-29"      # last column to use for features one year before target
TARGET_COL = "2025-02-28"      # target column to predict


In [2]:
# helper functions for trimming, growth, and volatility

def get_zhvi_columns(df):
    return df.columns[df.columns.str.match(r'^\d{4}-\d{2}-\d{2}$')]

def longest_nan_streak(row):
    is_null = row.isnull()
    max_streak = 0
    current_streak = 0
    for val in is_null:
        if val:
            current_streak += 1
            max_streak = max(max_streak, current_streak)
        else:
            current_streak = 0
    return max_streak

def trim_to_first_valid(series):
    first_valid = series.first_valid_index()
    if first_valid:
        return series.loc[first_valid:]
    return pd.Series(dtype='float64')

# calculates percent change from n months ago
def yoy_growth(series, months_back=12):
    if len(series) < months_back + 1 or series.iloc[-months_back - 1] == 0:
        return np.nan
    return 100 * (series.iloc[-1] - series.iloc[-months_back - 1]) / series.iloc[-months_back - 1]

# calculates total percent growth over entire series
def total_growth(series):
    if len(series) < 2 or series.iloc[0] == 0:
        return np.nan
    return 100 * (series.iloc[-1] - series.iloc[0]) / series.iloc[0]

# calculates compound annual growth rate
def cagr(series):
    if len(series) < 2 or series.iloc[0] <= 0:
        return np.nan
    years = len(series) / 12
    return ((series.iloc[-1] / series.iloc[0]) ** (1 / years) - 1) * 100

# calculates average monthly percent change
def avg_monthly_growth(series):
    monthly_pct = series.pct_change()
    return 100 * monthly_pct.mean()

# calculates standard deviation of monthly percent changes
def volatility(series):
    monthly_pct = series.pct_change()
    return 100 * monthly_pct.std()

# calculates yoy growth for each full year with available data
def compute_yoy_by_year(series):
    series = series.copy()
    series.index = pd.to_datetime(series.index)
    yearly_growth = {}

    for year in range(series.index.year.min() + 1, series.index.year.max() + 1):
        try:
            prev = series[series.index.year == year - 1].iloc[-1]
            curr = series[series.index.year == year].iloc[-1]
            if pd.notna(prev) and prev != 0:
                growth = 100 * (curr - prev) / prev
                yearly_growth[year] = growth
        except IndexError:
            continue

    return pd.Series(yearly_growth)


In [3]:
# loads dataset, fills metadata gaps, trims zhvi, interpolates missing values

df = pd.read_csv(DATA_PATH)
df = df[df["State"] == TARGET_STATE].copy()
print(f"loaded {df.shape[0]} ZIPs from {TARGET_STATE}")

zhvi_cols = get_zhvi_columns(df)

# check initial missing metadata counts
meta_cols = ["RegionID", "RegionName", "City", "Metro", "CountyName", "SizeRank", "State"]
missing_meta = df[meta_cols].isnull().sum()
missing_meta = missing_meta[missing_meta > 0]
both_missing = df[df["City"].isnull() & df["Metro"].isnull()].shape[0]
only_city_missing = df[df["City"].isnull() & df["Metro"].notnull()].shape[0]
only_metro_missing = df[df["Metro"].isnull() & df["City"].notnull()].shape[0]

print("\nmissing metadata breakdown:")
print(f"missing both city and metro: {both_missing}")
print(f"missing only city: {only_city_missing}")
print(f"missing only metro: {only_metro_missing}")

# fill metro using city → metro mapping, avoiding ambiguous cities
ambiguous_city_set = {"Boston", "Tifton"}
safe_city_to_metro = (
    df[df["City"].notna() & df["Metro"].notna() & ~df["City"].isin(ambiguous_city_set)]
    .groupby("City")["Metro"]
    .agg(lambda x: x.mode().iloc[0])
    .to_dict()
)

df["Metro"] = df.apply(
    lambda row: safe_city_to_metro.get(row["City"], row["Metro"]) if pd.isna(row["Metro"]) else row["Metro"],
    axis=1
)

# fill remaining metro using county to metro mapping
county_to_metro = (
    df[df["Metro"].notna()][["CountyName", "Metro"]]
    .drop_duplicates()
    .groupby("CountyName")["Metro"]
    .agg(lambda x: x.mode().iloc[0])
    .to_dict()
)

df["Metro"] = df.apply(
    lambda row: county_to_metro.get(row["CountyName"], row["Metro"]) if pd.isna(row["Metro"]) else row["Metro"],
    axis=1
)

# drop rows missing both city and metro, then drop rows missing either
df = df[~(df["City"].isnull() & df["Metro"].isnull())].copy()
before_drop = df.shape[0]
df = df[df["City"].notna() & df["Metro"].notna()].copy()
after_drop = df.shape[0]

print(f"\nmetadata cleanup:")
print(f"rows before drop: {before_drop}")
print(f"rows after drop: {after_drop}")
print(f"rows removed: {before_drop - after_drop}")

# ========== ✅ NEW: calculate YoY_2025 BEFORE trimming ==========
def compute_target_yoy(row, cutoff=CUTOFF_COL, target=TARGET_COL):
    try:
        prev = row[cutoff]
        curr = row[target]
        if pd.notna(prev) and prev != 0:
            return 100 * (curr - prev) / prev
    except KeyError:
        pass
    return np.nan

df["YoY_2025"] = df.apply(lambda row: compute_target_yoy(row), axis=1)

# trim zhvi series to first valid value and up to cutoff
cutoff_cols = [col for col in zhvi_cols if col <= CUTOFF_COL]
trimmed_series_list, trimmed_lengths, missing_counts = [], [], []

for _, row in df.iterrows():
    trimmed = trim_to_first_valid(row[cutoff_cols]).astype("float64")
    trimmed_series_list.append(trimmed)
    trimmed_lengths.append(len(trimmed))
    missing_counts.append(trimmed.isnull().sum())

df["TrimmedZHVI"] = trimmed_series_list
df["TrimmedLength"] = trimmed_lengths
df["TrimmedMissing"] = missing_counts

print(f"\ntrimmed zhvi series for {len(df)} ZIPs")

# interpolate missing values within each trimmed series
interpolated_series, interpolated_missing_counts = [], []

for s in df["TrimmedZHVI"]:
    interpolated = s.interpolate(method="linear", limit_direction="both")
    interpolated_series.append(interpolated)
    interpolated_missing_counts.append(interpolated.isna().sum())

df["InterpolatedZHVI"] = interpolated_series
df["InterpolatedMissing"] = interpolated_missing_counts

before_interp = df.shape[0]
df = df[df["InterpolatedMissing"] == 0].copy()
after_interp = df.shape[0]

print(f"\ninterpolation result:")
print(f"clean rows: {after_interp}")
print(f"removed rows: {before_interp - after_interp}")


loaded 665 ZIPs from GA

missing metadata breakdown:
missing both city and metro: 11
missing only city: 25
missing only metro: 111

metadata cleanup:
rows before drop: 658
rows after drop: 558
rows removed: 100

trimmed zhvi series for 558 ZIPs

interpolation result:
clean rows: 558
removed rows: 0


In [4]:
# computes engineered features using interpolated zhvi

# final zhvi is the value at the cutoff
df["FinalZHVI"] = df["InterpolatedZHVI"].apply(lambda s: s.loc[CUTOFF_COL] if CUTOFF_COL in s.index else np.nan)

# z-score across all final values
df["FinalZScore"] = zscore(df["FinalZHVI"])

# assign value tier using asymmetric z-score bins
def zhvi_asymmetric_tier(z):
    if z >= 2.5:
        return "very high"
    elif z >= 1.5:
        return "high"
    elif z >= -0.5:
        return "mid"
    elif z >= -1.0:
        return "low"
    else:
        return "very low"

df["ZHVI_Tier"] = df["FinalZScore"].apply(zhvi_asymmetric_tier)

# compute yoy growth per year (EXCLUDING 2025 to avoid leakage)
yoy_df = df["InterpolatedZHVI"].apply(lambda s: compute_yoy_by_year(s.loc[:CUTOFF_COL]))
yoy_df = yoy_df.add_prefix("YoY_")
df = pd.concat([df, yoy_df], axis=1)

# compute aggregate features from yearly growth
df["AvgYoYGrowth"] = yoy_df.mean(axis=1)
df["MedianYoYGrowth"] = yoy_df.median(axis=1)
df["YoYGrowthVolatility"] = yoy_df.std(axis=1)
df["NegativeGrowthYears"] = (yoy_df < 0).sum(axis=1)

# compute full-period features using the interpolated series
df["Total_Growth"] = df["InterpolatedZHVI"].apply(total_growth)
df["CAGR"] = df["InterpolatedZHVI"].apply(cagr)
df["AvgMonthlyGrowth"] = df["InterpolatedZHVI"].apply(avg_monthly_growth)
df["Volatility"] = df["InterpolatedZHVI"].apply(volatility)

# rename RegionName to ZIP for clarity
df.rename(columns={"RegionName": "ZIP"}, inplace=True)

# show distribution of value tiers
print("zhvi tier distribution:")
print(df["ZHVI_Tier"].value_counts().sort_index())


zhvi tier distribution:
ZHVI_Tier
high          22
low          128
mid          323
very high     18
very low      67
Name: count, dtype: int64


In [5]:
# ========== Pull and merge ACS ZIP-level features (Demographic, Housing, Education, etc.) ==========

import requests

API_KEY = "b06c64cd8eadc4fa292ce05e788144d98223fa3f"

# ACS variable codes to fetch
acs_variables = [
    # housing + vehicles
    "B25003_001E", "B25003_003E",     # total housing, renter-occupied
    "B08201_001E", "B08201_002E",     # total households, no vehicle

    # education
    "B15003_001E",                   # education total
    "B15003_022E", "B15003_023E", "B15003_024E", "B15003_025E",  # degrees

    # employment and poverty
    "B23025_003E", "B23025_005E",    # labor force, unemployed
    "B17001_001E", "B17001_002E",    # poverty total, below poverty

    # age breakdown
    "B01003_001E",  # total population
    "B01001_003E", "B01001_027E",  # age <5 (male/female)
    "B01001_004E", "B01001_028E",  # age 5-9
    "B01001_005E", "B01001_029E",  # age 10-14
    "B01001_006E", "B01001_030E",  # age 15-17
    "B01001_007E", "B01001_031E",  # age 18-19
    "B01001_008E", "B01001_032E",  # age 20-24
    "B01001_009E", "B01001_033E",  # age 25-29
    "B01001_010E", "B01001_034E",  # age 30-34
    "B01001_020E", "B01001_044E",  # age 65+

    # household types
    "B11001_001E", "B11001_008E",   # total households, 1-person
    "B11016_001E", "B11016_010E"    # total households, 4+ people
]

# build API URL
base_url = "https://api.census.gov/data/2023/acs/acs5"
get_vars = ",".join(["NAME"] + acs_variables)
acs_url = f"{base_url}?get={get_vars}&for=zip%20code%20tabulation%20area:*&key={API_KEY}"

# fetch and parse
response = requests.get(acs_url)
data = response.json()
columns = data[0]
rows = data[1:]
acs_df = pd.DataFrame(rows, columns=columns)

# rename for clarity
acs_df.rename(columns={
    "zip code tabulation area": "ZIP",
    "B25003_001E": "Total_Housing",
    "B25003_003E": "Renter_Occupied",
    "B08201_001E": "Total_Households_Vehicle",
    "B08201_002E": "No_Vehicle_Households",
    "B15003_001E": "Education_Total",
    "B15003_022E": "Bachelors",
    "B15003_023E": "Masters",
    "B15003_024E": "Professional",
    "B15003_025E": "Doctorate",
    "B23025_003E": "Labor_Force",
    "B23025_005E": "Unemployed",
    "B17001_001E": "Poverty_Total",
    "B17001_002E": "Below_Poverty",
    "B01003_001E": "Population_Total",
    "B01001_003E": "Age_U5_M",  "B01001_027E": "Age_U5_F",
    "B01001_004E": "Age_5_9_M", "B01001_028E": "Age_5_9_F",
    "B01001_005E": "Age_10_14_M", "B01001_029E": "Age_10_14_F",
    "B01001_006E": "Age_15_17_M", "B01001_030E": "Age_15_17_F",
    "B01001_007E": "Age_18_19_M", "B01001_031E": "Age_18_19_F",
    "B01001_008E": "Age_20_24_M", "B01001_032E": "Age_20_24_F",
    "B01001_009E": "Age_25_29_M", "B01001_033E": "Age_25_29_F",
    "B01001_010E": "Age_30_34_M", "B01001_034E": "Age_30_34_F",
    "B01001_020E": "Age_65p_M", "B01001_044E": "Age_65p_F",
    "B11001_001E": "Household_Total",
    "B11001_008E": "One_Person_Household",
    "B11016_001E": "Households_SizeTotal",
    "B11016_010E": "Households_4plus"
}, inplace=True)

# convert to numeric
for col in acs_df.columns:
    if col not in ["ZIP", "NAME"]:
        acs_df[col] = pd.to_numeric(acs_df[col], errors="coerce")

# engineered percentage features
acs_df["Pct_Renter_Occupied"] = 100 * acs_df["Renter_Occupied"] / acs_df["Total_Housing"]
acs_df["Pct_Bachelors_Or_Higher"] = 100 * (
    acs_df["Bachelors"] + acs_df["Masters"] + acs_df["Professional"] + acs_df["Doctorate"]
) / acs_df["Education_Total"]
acs_df["Unemployment_Rate"] = 100 * acs_df["Unemployed"] / acs_df["Labor_Force"]
acs_df["Pct_Below_Poverty"] = 100 * acs_df["Below_Poverty"] / acs_df["Poverty_Total"]
acs_df["Pct_No_Vehicle"] = 100 * acs_df["No_Vehicle_Households"] / acs_df["Total_Households_Vehicle"]
acs_df["Pct_One_Person_HH"] = 100 * acs_df["One_Person_Household"] / acs_df["Household_Total"]
acs_df["Pct_4plus_HH"] = 100 * acs_df["Households_4plus"] / acs_df["Households_SizeTotal"]
acs_df["Pct_Age_0_17"] = 100 * (
    acs_df["Age_U5_M"] + acs_df["Age_U5_F"] +
    acs_df["Age_5_9_M"] + acs_df["Age_5_9_F"] +
    acs_df["Age_10_14_M"] + acs_df["Age_10_14_F"] +
    acs_df["Age_15_17_M"] + acs_df["Age_15_17_F"]
) / acs_df["Population_Total"]
acs_df["Pct_Age_18_34"] = 100 * (
    acs_df["Age_18_19_M"] + acs_df["Age_18_19_F"] +
    acs_df["Age_20_24_M"] + acs_df["Age_20_24_F"] +
    acs_df["Age_25_29_M"] + acs_df["Age_25_29_F"] +
    acs_df["Age_30_34_M"] + acs_df["Age_30_34_F"]
) / acs_df["Population_Total"]
acs_df["Pct_Age_65plus"] = 100 * (
    acs_df["Age_65p_M"] + acs_df["Age_65p_F"]
) / acs_df["Population_Total"]

# final subset
final_acs = acs_df[[
    "ZIP", "Pct_Renter_Occupied", "Pct_Bachelors_Or_Higher", "Unemployment_Rate",
    "Pct_Below_Poverty", "Pct_No_Vehicle", "Pct_One_Person_HH", "Pct_4plus_HH",
    "Pct_Age_0_17", "Pct_Age_18_34", "Pct_Age_65plus"
]].copy()
final_acs["ZIP"] = final_acs["ZIP"].astype(str)

# merge into main df
df["ZIP"] = df["ZIP"].astype(str)
df = df.merge(final_acs, on="ZIP", how="left")

# check merge results
missing_zips = df["Pct_Renter_Occupied"].isna().sum()
print(f"✅ Merged ACS ZIP-level features. Missing ZIP matches: {missing_zips}")


✅ Merged ACS ZIP-level features. Missing ZIP matches: 0


In [6]:
# prepares final export dataset and writes to csv

# assign new state-specific rank based on original SizeRank values
df["StateSizeRank"] = df["SizeRank"].rank(method="min").astype(int)

# drop RegionID and original SizeRank
df.drop(columns=["RegionID", "SizeRank"], inplace=True)

# columns to drop
drop_cols = [
    col for col in df.columns
    if col in [
        "State", "StateName", "RegionType", "TrimmedZHVI", "InterpolatedZHVI",
        "TrimmedLength", "TrimmedMissing", "InterpolatedMissing",
        "Total_Growth", "CAGR", "AvgMonthlyGrowth", "Volatility", "City"
    ] or col in zhvi_cols
]

# create final cleaned version for modeling
export_df = df.drop(columns=drop_cols)

# reorder columns
meta_cols = ["ZIP", "Metro", "CountyName", "StateSizeRank"]
target_col = "YoY_2025"
feature_cols = [col for col in export_df.columns if col not in meta_cols + [target_col]]
ordered_cols = meta_cols + feature_cols + [target_col]
export_df = export_df[ordered_cols]

# shuffle rows
export_df = export_df.sample(frac=1, random_state=42).reset_index(drop=True)

# export to csv
print("final dataset preview:")
display(export_df.head(0))
export_df.to_csv("final_processed_dataset.csv", index=False)
print(f"exported cleaned dataset with {export_df.shape[0]} rows and {export_df.shape[1]} columns")


final dataset preview:


,ZIP,Metro,CountyName,StateSizeRank,FinalZHVI,FinalZScore,ZHVI_Tier,YoY_2001,YoY_2002,YoY_2003,...,Pct_Bachelors_Or_Higher,Unemployment_Rate,Pct_Below_Poverty,Pct_No_Vehicle,Pct_One_Person_HH,Pct_4plus_HH,Pct_Age_0_17,Pct_Age_18_34,Pct_Age_65plus,YoY_2025


exported cleaned dataset with 558 rows and 46 columns
